# What's happening here?

4_link_video_behavior.ipynb

Run after 1_, 2_, and 3_

Made in environment **PMS_Study**

## TO DO: bekijk file PMS_analysis/2_preparation

Probeer veel met loops en functions te werken om de code overzichtelijk te houden

 * Start with file cleanData.csv (geeft informatie over van welke participant welke files bestaan)
 * Load timings.csv file that Tilia and Sofie made
 * Start loop from the top and use Order and A-B
 * In loop check if file (corresponding CSV) exists in Directory (/10001 or /10002 depedent of A/B): later refered to as behaviordata
 * In loop check if file (corresponding .WEBM) exists in Directory
 
If both exist:
- Check if output of Openface exists in Z:\shares\ghepmk_data\2020_Kappen_PMS\10002\video_features (or /10001 dependent of A/B)
- IF True:
Load this as facedata
- IF False:
Print: "Video file present for file (insert filename) but no openface present)"
Break/skip

Use filename/pptnum to search for right starting time in timings.csv variable
- Find timing for beep.wav in behaviordata. 
- Set new zero point (link between the timings.csv point and this)
- Make mean scores on a few variables (we will expand this later) for the corresponding frames of each of the trials.
- To do this, you should write a function that takes the right points of a trial (start, end, trialnum etc) and save this data to a dataframe

Compute this in a way that you would be able to add it to allPMSdata.cvs

Add it all to a new dataframe

Write to csv

In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math
import time
import json
import statistics

import features as ft

from tqdm import tqdm
from pathlib import Path

import re

In [2]:
# Create all relevant directories
data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)

# Load base data
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")
allData = pd.read_csv(data_all_dir + "allPMSdata3.csv", encoding = "ISO-8859-1")
timings = pd.read_csv(data_all_dir + "timings.csv", encoding = "ISO-8859-1")
timings = timings.rename(columns={"ï»¿ImageGroup": "ImageGroup"}, index={'ONE': 'Row_1'}) # Rename first column, because was formatted weird

redoOpenface = 1 # Set to 1 if you want to re-do OpenFace processing. For instance if you added features, or changed methods

In [3]:
# Get all OpenFace files
os.chdir(data_dir)
AFiles = glob.glob("10001/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BFiles = glob.glob("10002/video_features/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory

# Opening JSON file with all video duration information
f = open(data_all_dir + 'video_durations.json',)
durationsdata = json.load(f)
f.close()

In [4]:
def get_openfacestuff(fileDir):
    print(fileDir)
    if len(indices) > 1:
        print("Too many OpenFace files for ppt: " + str(pptnum) + " at moment " + moment)
    else: # If we get to here - the party really starts
        try:
            math.isnan(allData['AU12'][idx[0]])
        except:
            allData["AU12"] = np.nan # In case this column doesn't exist yet, create a column with only nan's so th next if statement always passes
            
        if math.isnan(allData['AU12'][idx[0]]) or redoOpenface == 1: # Check if OpenFace values have already been added, of whether manually set at top of script to re do this (so it doesnt happen unneccessarily
            openface = pd.read_csv(fileDir, encoding = "ISO-8859-1") # Read openface output csv

            # Get corresponding video file duration 
            splits = fileDir.split('/video_features\\')
            fileduration = [value for key, value in durationsdata.items() if splits[0] in key.lower() and splits[1][23:len(splits[1])-4] in key.lower()] # Very weird formatting, but when checking for directory (10001/10002) and only the date (something with curly bracktes messes it up. 
            frameduration = fileduration[0] / len(openface)
            openface.insert (3, "timestamp2", np.arange(0, fileduration[0], frameduration).tolist()[0:len(openface)]) # Add new timestamps right after the first timestamp || We're taking [0:len(openface)] here because due to rounding error, the newly created timestamps might be one longer than th array. 
            videoOffset
            behaviorOffset = allData['startBeep'][idx[0]]
            correctedStarts = allData['startView'][idx] - behaviorOffset # startViews - offSet
            videoStarts = ((correctedStarts + videoOffset) / 1000).tolist() # These will be the timestamps in seconds because / 1000

            correctedEnds = allData['stopView'][idx] - behaviorOffset # startViews - offSet
            videoEnds = ((correctedEnds + videoOffset) / 1000).tolist()

            # Loop over all the trials and get the appropriate values - add to allData
            for trial in range(0,len(videoStarts)):
                vidStartIdx = openface.iloc[(openface['timestamp2']-videoStarts[trial]).abs().argsort()[:1]].index.tolist()[0] # Get start idx in openface
                vidEndIdx = openface.iloc[(openface['timestamp2']-videoEnds[trial]).abs().argsort()[:1]].index.tolist()[0] # Get end idx in openface

                # Create mean value between these idx's and add to big dataframe
                allData.loc[idx[trial], 'AU12'] = openface[' AU12_r'][vidStartIdx:vidEndIdx].mean() 
                allData.loc[idx[trial], 'AU15'] = openface[' AU15_r'][vidStartIdx:vidEndIdx].mean() 
                
                allData.loc[idx[trial], 'arousal'] = ft.compute_arousal(openface[vidStartIdx:vidEndIdx])['mean_Arousal']
                allData.loc[idx[trial], 'blinkrate'] = ft.compute_blink_rate(openface[vidStartIdx:vidEndIdx])
                allData.loc[idx[trial], 'PD_Mean'] = ft.compute_PD_features(openface[vidStartIdx:vidEndIdx])['mean_PD']
                allData.loc[idx[trial], 'PD_Std'] = ft.compute_PD_features(openface[vidStartIdx:vidEndIdx])['std_PD']
                allData.loc[idx[trial], 'PD_Max'] = ft.compute_PD_features(openface[vidStartIdx:vidEndIdx])['max_PD']
                
                allData.loc[idx[trial], 'headMovement'] = ft.compute_head_motion(openface[vidStartIdx:vidEndIdx])['compound_Motion']

                allData.loc[idx[trial], 'happy'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Happy']
                allData.loc[idx[trial], 'sad'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Sad']
                allData.loc[idx[trial], 'disgust'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Disgust']
                allData.loc[idx[trial], 'anger'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Angry']
                allData.loc[idx[trial], 'scared'] = ft.compute_emotions(openface[vidStartIdx:vidEndIdx])['mean_Scared']
                
                allData.loc[idx[trial], 'confidence'] = ft.compute_quality(openface[vidStartIdx:vidEndIdx])['mean_confidence']

In [ ]:
# The BIG loop where all the magic happens

index = 0
for pptnum in tqdm(timings["pptnumber"]): # Loop over all ppt's that had a video available (therefore present in timings.csv)
    if '514' not in str(pptnum) and '583' not in str(pptnum) and '342' not in str(pptnum) and '4888' not in str(pptnum): # '&': This is something that gives errors, '383': Corrupted file, '467': same, '495': same, '502': same

        print("processing pptnum: " + str(pptnum))
        idx = np.where(allData['ID'] == int(pptnum))[0]
        if timings["Exclusion"][index] == 1: # Only continue the loop if there is no Exclusion marked
            print("Exclusion file for ppt: " + str(pptnum) + " at moment: " + timings["ImageGroup"][index])
        else:
            videoOffset = float(timings["StartTime"][index])
            if videoOffset == 0: # if a timing start point was not present, it was set to 0. We don't process those because we cant sync them
                print('start time for this ppt is set to zero')
            else:
                if len(idx) < 1: # Check if occurences in datafile exist for this participant/video file
                    print("No input data found for video file of pptnum: " + str(pptnum) + " with video file " + timings["ImageGroup"][index] )
                else:           
                    moment = timings["ImageGroup"][index]
                    if moment ==  'A':
                        indices = [i for i, s in enumerate(AFiles) if str(pptnum) in s]
                        try:
                            fileDir = AFiles[indices[0]]
                        except:
                            print('this file is not found - video present, but no openface file for ' + str(pptnum))
                        else: 
                            get_openfacestuff(fileDir)
                    elif moment == 'B':
                        indices = [i for i, s in enumerate(BFiles) if str(pptnum) in s]
                        try:
                            fileDir = BFiles[indices[0]]
                        except:
                            print('this file is not found - video present, but no openface file for ' + str(pptnum))
                        else: 
                            get_openfacestuff(fileDir)

                    else:
                        print("Some error with testing moment")
                        break

    index += 1 # Create counter
#     if index > 10:
#         break

  0%|                                                                                          | 0/525 [00:00<?, ?it/s]

processing pptnum: 14
Exclusion file for ppt: 14 at moment: B
processing pptnum: 66
Exclusion file for ppt: 66 at moment: B
processing pptnum: 106
No input data found for video file of pptnum: 106 with video file B
processing pptnum: 119
No input data found for video file of pptnum: 119 with video file B
processing pptnum: 123
No input data found for video file of pptnum: 123 with video file B
processing pptnum: 124
No input data found for video file of pptnum: 124 with video file B
processing pptnum: 125
No input data found for video file of pptnum: 125 with video file B
processing pptnum: 126
No input data found for video file of pptnum: 126 with video file B
processing pptnum: 127
start time for this ppt is set to zero
processing pptnum: 129
10002/video_features\video_{ParticipantNo}_129_9-10-2020.csv


In [6]:
allData

,Unnamed: 0,ID,Age,FirstMenstrual,MenstrualDuration,Exclusie,TrueFollicular,TrueLuteal,Isala,Order,...,PD_Mean,PD_Std,PD_Max,headMovement,happy,sad,disgust,anger,scared,confidence
0,0,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,165,21,16,28,NaN,5-12-2020,19-12-2020,NaN,B-A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,15857,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15858,15858,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15859,15859,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15860,15860,535,32,13,23,NaN,7-5-2021,16-5-2021,03660,A-B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# os.chdir(os.path.join(data_dir,date_dir))
# allData.to_csv('allPMSdata3.csv')

In [6]:
allData.to_csv(str(data_dir) + '\\' + str(date_dir) + 'allPMSdata3.csv')